# Example: Hierarchical Control System

This notebook demonstrates a two-level hierarchical control system. This is a core concept of the Smart Water Platform, where high-level supervisory agents manage the objectives of low-level, local controllers.

The scenario consists of:
- **A low-level control loop**: A `LocalControlAgent` uses a PID controller to adjust a `Gate`'s opening to maintain a water level setpoint in a `Reservoir`.
- **A high-level supervisory loop**: A `CentralDispatcher` agent monitors the reservoir's water level. If the level exceeds a 'flood' threshold, it issues a new, lower setpoint to the local controller to bring the level down.

## Simulation Setup

To make this notebook self-contained, the setup code from the `helpers.py` module is included directly here. 

The system is initialized with the reservoir level at 19.0m. The initial setpoint for the local controller is 15.0m. The central dispatcher's rule is to change the setpoint to 12.0m if the water level exceeds 18.0m. We should therefore see the simulation start with the high-level agent immediately intervening.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.central_coordination.dispatch.central_dispatcher import CentralDispatcher
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.core_engine.testing.simulation_harness import SimulationHarness

def setup_hierarchical_control_system(message_bus, simulation_dt):
    RESERVOIR_STATE_TOPIC = "state.reservoir.level"
    GATE_STATE_TOPIC = "state.gate.gate_1"
    GATE_ACTION_TOPIC = "action.gate.opening"
    GATE_COMMAND_TOPIC = "command.gate1.setpoint"

    gate_params = {'max_rate_of_change': 0.5, 'discharge_coefficient': 0.6, 'width': 10, 'max_opening': 5.0}
    reservoir = Reservoir(name="reservoir_1", initial_state={'volume': 28.5e6, 'water_level': 19.0}, parameters={'surface_area': 1.5e6})
    gate = Gate(name="gate_1", initial_state={'opening': 0.1}, parameters=gate_params, message_bus=message_bus, action_topic=GATE_ACTION_TOPIC)

    reservoir_twin = DigitalTwinAgent(agent_id="twin_reservoir_1", simulated_object=reservoir, message_bus=message_bus, state_topic=RESERVOIR_STATE_TOPIC)
    gate_twin = DigitalTwinAgent(agent_id="twin_gate_1", simulated_object=gate, message_bus=message_bus, state_topic=GATE_STATE_TOPIC)

    pid = PIDController(Kp=-0.8, Ki=-0.1, Kd=-0.2, setpoint=15.0, min_output=0.0, max_output=gate_params['max_opening'])
    lca = LocalControlAgent(agent_id="lca_gate_1", controller=pid, message_bus=message_bus, observation_topic=RESERVOIR_STATE_TOPIC, observation_key='water_level', action_topic=GATE_ACTION_TOPIC, dt=simulation_dt, command_topic=GATE_COMMAND_TOPIC, feedback_topic=GATE_STATE_TOPIC)

    rules = {'flood_threshold': 18.0, 'normal_setpoint': 15.0, 'flood_setpoint': 12.0}
    dispatcher = CentralDispatcher(agent_id="dispatcher_1", message_bus=message_bus, state_subscriptions={'reservoir_level': RESERVOIR_STATE_TOPIC}, command_topics={'gate1_command': GATE_COMMAND_TOPIC}, rules=rules)

    return [reservoir, gate], [reservoir_twin, gate_twin, lca, dispatcher]

# Main simulation logic
message_bus = MessageBus()
simulation_dt = 1.0
components, agents = setup_hierarchical_control_system(message_bus, simulation_dt)

simulation_config = {'duration': 500, 'dt': simulation_dt}
harness = SimulationHarness(config=simulation_config)

for component in components:
    harness.add_component(component)
for agent in agents:
    harness.add_agent(agent)

harness.add_connection("reservoir_1", "gate_1")

# Build and run the MAS simulation
import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    harness.build()
    harness.run_mas_simulation()
sys.stdout = original_stdout

print("Hierarchical control simulation complete.")

## Results and Visualization

The plots below show the water level of the reservoir and the opening of the control gate. We expect to see the water level start at 19.0m and be driven down towards the 'flood' setpoint of 12.0m by the controller.

In [ ]:
# Extract data from history
time = [h['time'] for h in harness.history]
reservoir_levels = [h['reservoir_1']['water_level'] for h in harness.history]
gate_openings = [h['gate_1']['opening'] for h in harness.history]

# Create a DataFrame
df = pd.DataFrame({
    'Time': time,
    'Reservoir Level': reservoir_levels,
    'Gate Opening': gate_openings
})

print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Reservoir Level
ax1.plot(df['Time'], df['Reservoir Level'], label='Reservoir Water Level')
ax1.axhline(y=18.0, color='orange', linestyle='--', label='Flood Threshold (18.0m)')
ax1.axhline(y=15.0, color='green', linestyle='--', label='Normal Setpoint (15.0m)')
ax1.axhline(y=12.0, color='red', linestyle='--', label='Flood Setpoint (12.0m)')
ax1.set_ylabel('Water Level (m)')
ax1.set_title('Hierarchical Control Simulation')
ax1.grid(True)
ax1.legend()

# Gate Opening
ax2.plot(df['Time'], df['Gate Opening'], label='Gate 1 Opening', color='purple')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Gate Opening (m)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()